# Advent of Code 2024 Day 17 

### Setup

In [239]:
from aocd import get_data, submit

day = 17
year = 2024


In [240]:
with open('example.txt', 'r') as file:
    raw_sample_data = "".join(file.readlines())

raw_sample_data[:100]

'Register A: 729\nRegister B: 0\nRegister C: 0\n\nProgram: 0,1,5,4,3,0'

In [241]:
raw_test_data = get_data(day=day, year=year)

raw_test_data[:]

'Register A: 21539243\nRegister B: 0\nRegister C: 0\n\nProgram: 2,4,1,3,7,5,1,5,0,3,4,1,5,5,3,0'

##### Data Parsing

In [242]:
def parse_data(raw_data):
    lines = raw_data.splitlines()
    
    data = {'instructions': [], 'registers': {}}
    for line in lines[:3]:
        if 'A' in line:
            data['registers']['A'] = int(line.replace('Register A: ', ''))
        
        if 'B' in line:
            data['registers']['B'] = int(line.replace('Register B: ', ''))
            
        if 'C' in line:
            data['registers']['C'] = int(line.replace('Register C: ', ''))
        
    data['instructions'] = [int(x.strip()) for x in lines[-1].replace('Program:' , '').split(',')]
    
    return data
            

sample_data = parse_data(raw_sample_data)
test_data = parse_data(raw_test_data)

sample_data

{'instructions': [0, 1, 5, 4, 3, 0], 'registers': {'A': 729, 'B': 0, 'C': 0}}

### Part One!

In [243]:
use_sample_data = False
part = 'a'

In [244]:
data = sample_data if use_sample_data else test_data
data

{'instructions': [2, 4, 1, 3, 7, 5, 1, 5, 0, 3, 4, 1, 5, 5, 3, 0],
 'registers': {'A': 21539243, 'B': 0, 'C': 0}}

In [245]:
from typing import Dict

class Registers():
    
    def __init__(self, mappings: Dict):
        self._registers = mappings
        
    def __getitem__(self, key):
        return self._registers[key]
    
    def __setitem__(self, key, value):
        self._registers[key] = value
        
    def __repr__(self):
        return str(self._registers)
    
    def __str__(self):
        return str(self._registers)
    
    def __iter__(self):
        return iter(self._registers)
    
    def __len__(self):
        return len(self._registers)
    
    def __contains__(self, key):
        return key in self._registers
    
    def keys(self):
        return self._registers.keys()
    
    def values(self):
        return self._registers.values()
    
    def items(self):
        return self._registers.items()
    
    def copy(self):
        return Registers(self._registers.copy())

In [246]:
from typing import Union, Tuple, Any, List

OperationResult = Tuple[int, Registers, Any]

In [247]:
def get_combo(opcode, registers):
    if opcode <= 3:
        return opcode
    
    elif opcode == 4:
        return registers['A']
    
    elif opcode == 5:
        return registers['B']
    
    elif opcode == 6:
        return registers['C']
    
    else:
        return -1

In [248]:
def get_literal(opcode, registers):
    return opcode

In [249]:
import math

def adv(operand:int, registers:Registers, pointer:int) -> OperationResult:
    combo = get_combo(operand, registers)
    registers['A'] //= 2 ** combo
    
    return (pointer + 2, registers, None)

In [250]:
def bxl(operand:int, registers:Registers, pointer:int) -> OperationResult:
    literal = get_literal(operand, registers)
    registers['B'] ^= literal
    
    return (pointer + 2, registers, None)

In [251]:
def bst(operand:int, registers:Registers, pointer:int) -> OperationResult:
    combo = get_combo(operand, registers)
    registers['B'] = combo % 8
    
    return (pointer + 2, registers, None)

In [252]:
def jnz(operand:int, registers:Registers, pointer:int) -> OperationResult:
    if registers['A']:
        return (get_literal(operand, registers), registers, None)
    
    return (pointer + 2, registers, None)

In [253]:
def bxc(operand:int, registers:Registers, pointer:int) -> OperationResult:
    registers['B'] ^= registers['C']
    
    return (pointer + 2, registers, None)

In [254]:
def out(operand:int, registers:Registers, pointer:int) -> OperationResult:
    combo = get_combo(operand, registers)
    
    return (pointer + 2, registers, combo % 8)

In [255]:
def bdv(operand:int, registers:Registers, pointer:int) -> OperationResult:
    combo = get_combo(operand, registers)
    registers['B'] = registers['A'] // 2 ** combo
    
    return (pointer + 2, registers, None)

In [256]:
def cdv(operand:int, registers:Registers, pointer:int) -> OperationResult:
    combo = get_combo(operand, registers)
    registers['C'] = registers['A'] // 2 ** combo
    
    return (pointer + 2, registers, None)

In [257]:
from typing import Callable

def get_operation(opcode:int) -> Callable:
    if opcode == 0:
        return adv
    
    elif opcode == 1:
        return bxl
    
    elif opcode == 2:
        return bst
    
    elif opcode == 3:
        return jnz
    
    elif opcode == 4:
        return bxc
    
    elif opcode == 5:
        return out
    
    elif opcode == 6:
        return bdv
    
    elif opcode == 7:
        return cdv
    

In [258]:
def perform_instruction(opcode:int, operand:int, registers:Registers, pointer:int) -> OperationResult:
    operation = get_operation(opcode)
    
    return operation(operand, registers, pointer)

In [259]:
def run_program(instructions:List[str], registers:Registers) -> Registers:
    pointer = 0
    
    outputs = []
    while pointer < len(instructions):
        opcode, operand = instructions[pointer:pointer+2]
        
        pointer, registers, output = perform_instruction(opcode, operand, registers, pointer)
        
        if output is not None:
            outputs.append(output)
    
    return (registers, outputs)

In [260]:
instructions = data['instructions']
starting_registers = Registers(data['registers'])

registers, output = run_program(instructions, starting_registers.copy())

part_a_answer = ",".join([str(x) for x in output])
part_a_answer, use_sample_data

('6,7,5,2,1,3,5,1,7', False)

In [261]:
if not use_sample_data and part == 'a':
    submit(answer=part_a_answer, part='a', day=day, year=year, reopen=True)

That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


### Part Two!

In [275]:
use_sample_data = True
part='b'

In [276]:
data = sample_data if use_sample_data else test_data

In [ ]:
def find_program_copy_locations(instructions:List[int], Registers:Registers) -> List[int]:
    

In [ ]:
part_b_answer = 0

In [ ]:
if not use_sample_data and part == 'b':
    submit(answer=part_b_answer, part='b', day=day, year=year, reopen=True)